In [6]:
from timm import list_models, create_model
import torch
import torch.nn as nn

In [7]:
m = create_model('vit_base_patch16_siglip_224.v2_webli',pretrained=True,num_classes=0,global_pool='')

Unexpected keys (attn_pool.kv.bias, attn_pool.kv.weight, attn_pool.latent, attn_pool.mlp.fc1.bias, attn_pool.mlp.fc1.weight, attn_pool.mlp.fc2.bias, attn_pool.mlp.fc2.weight, attn_pool.norm.bias, attn_pool.norm.weight, attn_pool.proj.bias, attn_pool.proj.weight, attn_pool.q.bias, attn_pool.q.weight) found while loading pretrained weights. This may be expected if model is being adapted.


In [8]:
m(torch.rand(1,3,224,224)).shape

torch.Size([1, 196, 768])

In [9]:
def pixel_shuffle(x, scale_factor=2):
    bsz, seq, embed_dim = x.size()
    height = width = int(seq**0.5)
    x = x.view(bsz, height, width, embed_dim)
    x = x.view(bsz, height, int(width / scale_factor), embed_dim * scale_factor)
    x = x.permute(0, 2, 1, 3)
    x = x.reshape(bsz, int(width / scale_factor), int(height / scale_factor), embed_dim * (scale_factor**2))
    x = x.permute(0, 2, 1, 3)
    x = x.reshape(bsz, int(seq / (scale_factor**2)), embed_dim * (scale_factor**2))
    return x

In [10]:
pixel_shuffle(torch.rand(1,196,768)).shape

torch.Size([1, 49, 3072])

In [11]:
class VisionProjector(nn.Module):
    def __init__(self, dim, scale_factor):
        super().__init__()
        self.dim = dim
        self.scale_factor = scale_factor
        self.v_proj = nn.Linear(self.dim*(self.scale_factor**2),self.dim,bias=False)

    def pixel_shuffle(self, x):
        bsz, seq, embed_dim = x.size()
        height = width = int(seq**0.5)
        x = x.view(bsz, height, width, embed_dim)
        x = x.view(bsz, height, int(width / self.scale_factor), embed_dim * self.scale_factor)
        x = x.permute(0, 2, 1, 3)
        x = x.reshape(bsz, int(width / self.scale_factor), int(height / self.scale_factor), embed_dim * (self.scale_factor**2))
        x = x.permute(0, 2, 1, 3)
        x = x.reshape(bsz, int(seq / (self.scale_factor**2)), embed_dim * (self.scale_factor**2))
        return x

    def forward(self, x):
        batch, num_images, seq_len, dim = x.shape
        x = x.view(batch*num_images, seq_len, dim)
        x = self.pixel_shuffle(x)
        x = self.v_proj(x)
        x = x.view(batch, num_images, x.size(1), dim)
        return x

In [36]:
vp = VisionProjector(768,2)
images = vp(torch.rand(1,3,196,768))
images.shape

torch.Size([1, 3, 49, 768])

In [17]:
images2 = torch.unbind(images,1)
[i.shape for i in images2]

[torch.Size([1, 49, 768]), torch.Size([1, 49, 768]), torch.Size([1, 49, 768])]

In [14]:
import torch

In [33]:
inputs = torch.randint(0,2000,(1,300)).long()
input_embeddings = torch.zeros(1,300,768)
positions = [*list(range(2,2+49)),*list(range(88,88+49)),*list(range(230,230+49))]
len(positions)

147

In [34]:
inputs[:,positions] = 5678
image_mask = inputs == 5678

In [37]:
images = images.flatten(1,2)
images.shape

torch.Size([1, 147, 768])

In [38]:
input_embeddings[:,positions,:] = images

In [41]:
print(positions)

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278]
